## Zadania do wykonania

### Zadanie 1

Napisz dekorator, który autoryzuje użytkownika na podstawie podanego loginu i hasła. Do przechowywania danych należy wykorzystać klasę `shelve`. Do maskowania hasła można użyć następującej funkcji:

In [4]:
import hashlib, binascii
from os import urandom

def hash_password(password):
    """Hash a password for storing."""
    salt = hashlib.sha256(urandom(60)).hexdigest().encode('ascii')
    pwdhash = hashlib.pbkdf2_hmac('sha512', password.encode('utf-8'),
                                salt, 100000)
    pwdhash = binascii.hexlify(pwdhash)
    return (salt + pwdhash).decode('ascii')

hash_password('test')

'48920eeebdf7c90e19268e5553678278a394dbd2eb8664631614c4b184f306877544f1031388e80da7b1b4adcda3d6b8ed5ba433aa9de375fff65163c62e6d52a3796e6c5f183645eafd446879fef6f0f966416d98a4626f026930c70c417f6e'

Należy pamiętać, że `salt` należy utworzyć raz i zachować w chronionym pliku.

### Zadanie 2

Napisz dekorator, który ogranicza argument funkcji do zadanego przedziału.

### Zadanie 3

Napisz generator, który dla zadanego wielomianu i przedziału wartości, zwróci jego miejsca zerowe. Dla pierwszego wywołania wynikiem ma być pierwsze miejsce zerowe, dla drugiego, drugie itd. Można zastosować naiwny algorytm sprawdzający kolejne wartości wielomianu. Można również zastosować kodowanie wielomianu bezpośrednio w kodzie języka Python: `x**n*x**(n-1)`.

In [1]:
# zad 1
import shelve
import hashlib, binascii
from os import urandom


def hash_password(password, x):
    """Hash a password for storing."""
    pwdhash = hashlib.pbkdf2_hmac('sha512', password.encode('utf-8'), x, 100000)
    pwdhash = binascii.hexlify(pwdhash)
    return (x + pwdhash).decode('ascii')


def DEK(func):
    def wrapper(*args):
        file = shelve.open('haslo.db')
        pwd = file['password']
        salt = file['salt']
        file.close()
        if hash_password(args[1], salt) == hash_password(pwd, salt):
            args = (args[0], args[1])
        else:
            args = (args[0], hash_password(args[1], salt))
        func(*args)

    return wrapper


@DEK
def log(login, pwd):
    ret = "Logowanie nie powiodło się :("
    if (login == login and pwd == pwd):
        ret = "Udało się zalogować!"
    return print(ret)

login = "FDolas"
pwd = "@!#dolas$%"

file = shelve.open('haslo.db')
salt = hashlib.sha256(urandom(30)).hexdigest().encode('ascii')
file['salt'] = salt
file['password'] = pwd
file.close()

log("FDolas", "@!#dolas$%")

Udało się zalogować!


In [2]:
'''zad2
Napisz dekorator, który ogranicza argument funkcji do zadanego przedziału.'''


def dekorator(function):
    limit = 75

    def wrap(value):
        if value > limit:
            value = limit
        elif value < 0:
            value = 0
        function(value)

    return wrap


@dekorator
def numberLimit(number):
    print(number)


numberLimit(115)
numberLimit(72)
numberLimit(-1)

75
72
0


In [3]:
'''zad 3
Napisz generator, który dla zadanego wielomianu i przedziału wartości, zwróci jego miejsca zerowe.
Dla pierwszego wywołania wynikiem ma być pierwsze miejsce zerowe, dla drugiego, drugie itd.
Można zastosować naiwny algorytm sprawdzający kolejne wartości wielomianu.
Można również zastosować kodowanie wielomianu bezpośrednio w kodzie języka Python
x**n*x**(n-1)'''

def polynomialZerosRange(pD, ft, start, end):
    x = start
    while x <= end:
        y = 0
        for i in range(len(pD)):
            y += (pD[i] * (x ** ft[i]))
        if round(y,2) == 0:
            yield round(x,2)
            x += 0.1
        x += 0.001


powerD = [2, 5, -3, -1]

# 2x**3 + 5x**2 - 3x**1 - 1x**0
for i in polynomialZerosRange(powerD, [3, 2, 1, 0], -30, 30):
    print(f"Miejsca zerowe funkcji : x = {i} ")


Miejsca zerowe funkcji : x = -2.95 
Miejsca zerowe funkcji : x = -0.24 
Miejsca zerowe funkcji : x = 0.7 
